In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Codes de répétition

*Estimation d'utilisation : moins d'1 minute sur un processeur Heron (REMARQUE : il s'agit uniquement d'une estimation. Votre temps d'exécution peut varier.)*

## Contexte
Pour permettre la correction d'erreur quantique (QEC) en temps réel, vous devez être en mesure de contrôler dynamiquement le flux d'un programme quantique pendant son exécution, afin que les portes quantiques puissent être conditionnées par les résultats de mesure. Ce tutoriel exécute le code de retournement de bit (bit-flip code), qui est une forme très simple de QEC. Il présente un circuit quantique dynamique capable de protéger un qubit encodé contre une erreur de retournement de bit unique, puis évalue les performances du code de retournement de bit.

Vous pouvez exploiter des qubits ancillaires supplémentaires et l'intrication pour mesurer des *stabilisateurs* qui ne transforment pas l'information quantique encodée, tout en vous informant de certaines classes d'erreurs susceptibles de s'être produites. Un code stabilisateur quantique encode $k$ qubits logiques dans $n$ qubits physiques. Les codes stabilisateurs se concentrent de manière cruciale sur la correction d'un ensemble discret d'erreurs avec le support du groupe de Pauli $\Pi^n$.

Pour plus d'informations sur la QEC, consultez [Quantum Error Correction for Beginners.](https://arxiv.org/abs/0905.2794)
## Prérequis
Avant de commencer ce tutoriel, assurez-vous d'avoir installé les éléments suivants :

- Qiskit SDK v2.0 ou ultérieur, avec le support de la [visualisation](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 ou ultérieur (`pip install qiskit-ibm-runtime`)
## Configuration

In [ ]:
# Qiskit imports
from qiskit import (
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
)

# Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

from qiskit_ibm_runtime.circuit import MidCircuitMeasure

service = QiskitRuntimeService()

## Étape 1. Transposer les entrées classiques en un problème quantique

### Construire un circuit stabilisateur de retournement de bit

Le code de retournement de bit est l'un des exemples les plus simples de code stabilisateur. Il protège l'état contre une erreur de retournement de bit (X) unique sur l'un des qubits d'encodage. En considérant l'action de l'erreur de retournement de bit $X$, qui effectue la transformation $|0\rangle \rightarrow |1\rangle$ et $|1\rangle \rightarrow |0\rangle$ sur n'importe lequel de nos qubits, nous avons $\epsilon = {E_0, E_1, E_2 } = {IIX, IXI, XII}$. Le code nécessite cinq qubits : trois sont utilisés pour encoder l'état protégé, et les deux restants servent d'ancillas pour la mesure des stabilisateurs.

In [ ]:
# Choose the least busy backend that supports `measure_2`.

backend = service.least_busy(
    filters=lambda b: "measure_2" in b.supported_instructions,
    operational=True,
    simulator=False,
    dynamic_circuits=True,
)

In [ ]:
qreg_data = QuantumRegister(3)
qreg_measure = QuantumRegister(2)
creg_data = ClassicalRegister(3, name="data")
creg_syndrome = ClassicalRegister(2, name="syndrome")
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]


def build_qc():
    """Build a typical error correction circuit"""
    return QuantumCircuit(qreg_data, qreg_measure, creg_data, creg_syndrome)


def initialize_qubits(circuit: QuantumCircuit):
    """Initialize qubit to |1>"""
    circuit.x(qreg_data[0])
    circuit.barrier(qreg_data)
    return circuit


def encode_bit_flip(circuit, state, ancillas) -> QuantumCircuit:
    """Encode bit-flip. This is done by simply adding a cx"""
    for ancilla in ancillas:
        circuit.cx(state, ancilla)
    circuit.barrier(state, *ancillas)
    return circuit


def measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_measure):
    """
    Measure the syndrome by measuring the parity.
    We reset our ancilla qubits after measuring the stabilizer
    so we can reuse them for repeated stabilizer measurements.
    Because we have already observed the state of the qubit,
    we can write the conditional reset protocol directly to
    avoid another round of qubit measurement if we used
    the `reset` instruction.
    """
    circuit.cx(qreg_data[0], qreg_measure[0])
    circuit.cx(qreg_data[1], qreg_measure[0])
    circuit.cx(qreg_data[0], qreg_measure[1])
    circuit.cx(qreg_data[2], qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure)
    circuit.append(MidCircuitMeasure(), [qreg_measure[0]], [creg_measure[0]])
    circuit.append(MidCircuitMeasure(), [qreg_measure[1]], [creg_measure[1]])

    with circuit.if_test((creg_measure[0], 1)):
        circuit.x(qreg_measure[0])
    with circuit.if_test((creg_measure[1], 1)):
        circuit.x(qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit


def apply_correction_bit(circuit, qreg_data, creg_syndrome):
    """We can detect where an error occurred and correct our state"""
    with circuit.if_test((creg_syndrome, 3)):
        circuit.x(qreg_data[0])
    with circuit.if_test((creg_syndrome, 1)):
        circuit.x(qreg_data[1])
    with circuit.if_test((creg_syndrome, 2)):
        circuit.x(qreg_data[2])
    circuit.barrier(qreg_data)
    return circuit


def apply_final_readout(circuit, qreg_data, creg_data):
    """Read out the final measurements"""
    circuit.barrier(qreg_data)
    circuit.measure(qreg_data, creg_data)
    return circuit

In [ ]:
def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:
    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit = measure_syndrome_bit(
        circuit, qreg_data, qreg_measure, creg_syndrome
    )

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit, qreg_data, creg_data)
    return circuit


circuit = build_error_correction_sequence(apply_correction=True)
circuit.draw(output="mpl", style="iqp", cregbundle=False)

<Image src="../docs/images/tutorials/repetition-codes/extracted-outputs/dbe02949-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/repetition-codes/extracted-outputs/dbe02949-1.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/repetition-codes/extracted-outputs/dbe02949-0.avif)

![Sortie de la cellule de code précédente](../docs/images/tutorials/repetition-codes/extracted-outputs/dbe02949-1.avif)

## Étape 2. Optimiser le problème pour l'exécution quantique
Pour réduire le temps total d'exécution des tâches, les primitives Qiskit n'acceptent que des circuits et des observables conformes aux instructions et à la connectivité prises en charge par le système cible (appelés circuits et observables ISA, pour « instruction set architecture »).  [En savoir plus sur la transpilation.](/guides/transpile)
### Générer des circuits ISA

In [6]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

isa_circuit.draw("mpl", style="iqp", idle_wires=False)

<Image src="../docs/images/tutorials/repetition-codes/extracted-outputs/67b55eef-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/repetition-codes/extracted-outputs/67b55eef-1.avif" alt="Output of the previous code cell" />

In [7]:
no_correction_circuit = build_error_correction_sequence(
    apply_correction=False
)

isa_no_correction_circuit = pm.run(no_correction_circuit)

## Étape 3. Exécuter à l'aide des primitives Qiskit
Exécutez la version avec correction appliquée et une version sans correction.

In [ ]:
sampler_no_correction = Sampler(backend)
job_no_correction = sampler_no_correction.run(
    [isa_no_correction_circuit], shots=1000
)
result_no_correction = job_no_correction.result()[0]

In [ ]:
sampler_with_correction = Sampler(backend)

job_with_correction = sampler_with_correction.run([isa_circuit], shots=1000)
result_with_correction = job_with_correction.result()[0]

In [10]:
print(f"Data (no correction):\n{result_no_correction.data.data.get_counts()}")
print(
    f"Syndrome (no correction):\n{result_no_correction.data.syndrome.get_counts()}"
)

Data (no correction):
{'111': 878, '011': 42, '110': 35, '101': 40, '100': 1, '001': 2, '000': 2}
Syndrome (no correction):
{'00': 942, '10': 33, '01': 22, '11': 3}


In [11]:
print(f"Data (corrected):\n{result_with_correction.data.data.get_counts()}")
print(
    f"Syndrome (corrected):\n{result_with_correction.data.syndrome.get_counts()}"
)

Data (corrected):
{'111': 889, '110': 25, '000': 11, '011': 45, '101': 17, '010': 10, '001': 2, '100': 1}
Syndrome (corrected):
{'00': 929, '01': 39, '10': 20, '11': 12}


## Step 4. Post-process, return result in classical format

You can see that the bit flip code detected and corrected many errors, resulting in fewer errors overall.

In [12]:
def decode_result(data_counts, syndrome_counts):
    shots = sum(data_counts.values())
    success_trials = data_counts.get("000", 0) + data_counts.get("111", 0)
    failed_trials = shots - success_trials
    error_correction_events = shots - syndrome_counts.get("00", 0)
    print(
        f"Bit flip errors were detected/corrected on {error_correction_events}/{shots} trials."
    )
    print(
        f"A final parity error was detected on {failed_trials}/{shots} trials."
    )

In [13]:
# non-corrected marginalized results
data_result = result_no_correction.data.data.get_counts()
marginalized_syndrome_result = result_no_correction.data.syndrome.get_counts()

print(
    f"Completed bit code experiment data measurement counts (no correction): {data_result}"
)
print(
    f"Completed bit code experiment syndrome measurement counts (no correction): {marginalized_syndrome_result}"
)
decode_result(data_result, marginalized_syndrome_result)

Completed bit code experiment data measurement counts (no correction): {'111': 878, '011': 42, '110': 35, '101': 40, '100': 1, '001': 2, '000': 2}
Completed bit code experiment syndrome measurement counts (no correction): {'00': 942, '10': 33, '01': 22, '11': 3}
Bit flip errors were detected/corrected on 58/1000 trials.
A final parity error was detected on 120/1000 trials.


In [14]:
# corrected marginalized results
corrected_data_result = result_with_correction.data.data.get_counts()
corrected_syndrome_result = result_with_correction.data.syndrome.get_counts()

print(
    f"Completed bit code experiment data measurement counts (corrected): {corrected_data_result}"
)
print(
    f"Completed bit code experiment syndrome measurement counts (corrected): {corrected_syndrome_result}"
)
decode_result(corrected_data_result, corrected_syndrome_result)

Completed bit code experiment data measurement counts (corrected): {'111': 889, '110': 25, '000': 11, '011': 45, '101': 17, '010': 10, '001': 2, '100': 1}
Completed bit code experiment syndrome measurement counts (corrected): {'00': 929, '01': 39, '10': 20, '11': 12}
Bit flip errors were detected/corrected on 71/1000 trials.
A final parity error was detected on 100/1000 trials.


## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_5onAlfA2Y7ac1FA)